# Mouse Facial Expressions - GPU 학습 (MPS) ✅ 수정됨

이 노트북은 Apple Silicon의 GPU(MPS)를 사용하여 모델을 학습합니다.

## 🚀 GPU 가속
- **MPS (Metal Performance Shaders)** 사용
- CPU 대비 **2-3배 빠른 학습 속도**
- 1 epoch: 약 15-20분 (CPU: 40-50분)
- 10 epochs: 약 3-4시간 (CPU: 7-8시간)

## ✅ 수정 사항 (2025-11-24)
- **MPS float64 에러 수정**: MPS는 float64를 지원하지 않으므로 모든 데이터를 float32로 변환
- **자동 GPU 감지**: `accelerator='auto'`로 설정하여 MPS 자동 사용

## Step 1: 환경 설정 및 GPU 확인

In [1]:
import os
import sys

# 프로젝트 디렉토리로 이동
os.chdir('/Users/minakang/Desktop/mouse-facial-expressions-2023-main')
print(f"현재 디렉토리: {os.getcwd()}")
print(f"Python 버전: {sys.version}")

현재 디렉토리: /Users/minakang/Desktop/mouse-facial-expressions-2023-main
Python 버전: 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]


In [2]:
# GPU 사용 가능 여부 확인
import torch
import lightning
import mlflow

print(f"PyTorch 버전: {torch.__version__}")
print(f"Lightning 버전: {lightning.__version__}")
print(f"MLflow 버전: {mlflow.__version__}")
print(f"\n=== GPU 상태 ===")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"MPS 사용 가능: {torch.backends.mps.is_available()}")
print(f"MPS 빌드됨: {torch.backends.mps.is_built()}")

if torch.backends.mps.is_available():
    print("\n✅ MPS (Apple GPU)를 사용할 수 있습니다!")
    print("   학습 속도가 CPU 대비 2-3배 빠릅니다.")
    
    # MPS 테스트
    try:
        test_tensor = torch.randn(10, 10).to('mps')
        print("   MPS 디바이스 테스트: ✅ 성공")
    except Exception as e:
        print(f"   MPS 디바이스 테스트: ❌ 실패 - {e}")
else:
    print("\n⚠️ MPS를 사용할 수 없습니다. CPU로 학습합니다.")

PyTorch 버전: 2.9.1
Lightning 버전: 2.5.6
MLflow 버전: 3.6.0

=== GPU 상태 ===
CUDA 사용 가능: False
MPS 사용 가능: True
MPS 빌드됨: True

✅ MPS (Apple GPU)를 사용할 수 있습니다!
   학습 속도가 CPU 대비 2-3배 빠릅니다.
   MPS 디바이스 테스트: ✅ 성공


## Step 2: 데이터 확인

In [3]:
# 사용 가능한 프레임 폴더 확인
import os

frames_folders = []
if os.path.exists('data/frames_backup'):
    frames_folders.append('data/frames_backup')
if os.path.exists('data/local_frames'):
    frames_folders.append('data/local_frames')

if frames_folders:
    print("✅ 사용 가능한 프레임 폴더:")
    for folder in frames_folders:
        frame_count = sum(1 for root, dirs, files in os.walk(folder) for f in files if f.endswith('.png'))
        print(f"  - {folder}: {frame_count:,} 프레임")
else:
    print("❌ 프레임 폴더를 찾을 수 없습니다.")

✅ 사용 가능한 프레임 폴더:
  - data/frames_backup: 57,174 프레임
  - data/local_frames: 57,174 프레임


In [4]:
# 데이터셋 파일 확인
import os

processed_dir = 'data/processed/task-1.1'
if os.path.exists(processed_dir):
    pkl_files = [f for f in os.listdir(processed_dir) if f.endswith('.pkl')]
    if len(pkl_files) > 0:
        print(f"✅ 데이터셋이 이미 존재합니다 ({len(pkl_files)}개 파일)")
    else:
        print("❌ 데이터셋 파일이 없습니다. Step 4를 실행하여 생성하세요.")
else:
    print("❌ 데이터셋 디렉토리가 없습니다. Step 4를 실행하여 생성하세요.")

✅ 데이터셋이 이미 존재합니다 (10개 파일)


## Step 3: .env 파일 설정

In [5]:
# 프레임 폴더 자동 감지 및 .env 파일 생성
import os

project_dir = '/Users/minakang/Desktop/mouse-facial-expressions-2023-main'

# 우선순위: frames_backup > local_frames
if os.path.exists(f'{project_dir}/data/frames_backup'):
    frames_folder = f'{project_dir}/data/frames_backup'
    print("✅ frames_backup 폴더를 사용합니다")
elif os.path.exists(f'{project_dir}/data/local_frames'):
    frames_folder = f'{project_dir}/data/local_frames'
    print("✅ local_frames 폴더를 사용합니다")
else:
    frames_folder = f'{project_dir}/data/local_frames'
    print("⚠️ 프레임 폴더를 찾을 수 없습니다. local_frames를 기본값으로 설정합니다.")

# .env 파일 생성
env_content = f"""MFE_RAW_VIDEO_FOLDER={project_dir}/data/raw
MFE_RAW_CSV_FOLDER={project_dir}/data/raw
MFE_VERSION=20230627
MFE_PROCESSED_VIDEO_FOLDER={project_dir}/data/processed
MFE_EXTRACTED_FRAMES_FOLDER={frames_folder}
MFE_TASKS={project_dir}/data/processed
"""

with open('.env', 'w') as f:
    f.write(env_content)

print("\n✅ .env 파일 생성 완료!")

✅ frames_backup 폴더를 사용합니다

✅ .env 파일 생성 완료!


## Step 4: 데이터셋 생성 (필요한 경우만 실행)

In [6]:
# 데이터셋이 없는 경우에만 실행
import os

processed_dir = 'data/processed/task-1.1'
if not os.path.exists(processed_dir) or len([f for f in os.listdir(processed_dir) if f.endswith('.pkl')]) == 0:
    print("데이터셋을 생성합니다...")
    !python mouse_facial_expressions/data/make_datasets.py task1 --version "1.1" --frameset_size 1
else:
    print("✅ 데이터셋이 이미 존재합니다. 이 단계를 건너뜁니다.")

✅ 데이터셋이 이미 존재합니다. 이 단계를 건너뜁니다.


## Step 5: GPU 학습 시작! 🚀

### Option A: 1 epoch 테스트 (~15-20분)

먼저 1 epoch로 테스트하여 GPU가 정상적으로 작동하는지 확인하세요.

In [7]:
# 1 epoch GPU 테스트
!python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 1 \
    --dataset_version "1.1"

2025-11-24 09:23:56,041 - __main__ - INFO - beginning model run train_task1_baseline_model.py
/opt/anaconda3/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025-11-24 09:23:56,645 - __main__ - INFO - Started mlflow crossvalidation run a0028fd77aad49a6a6bf1722bff2050c
2025-11-24 09:23:56,646 - __main__ - INFO - Logging parameters
{
    "epochs": 1,
    "dataset_version": "1.1",
    "model": 10,
    "learning_rate": 0.01,
    "train_batch_size": 20,
    "test_batch_size": 50,
    "warmup_steps": 0,
    "momentum": 0.9,
    "weight_decay": 0.0001,
    "warmup_decay": 0.0001,
    "label_smoothing": 0.1,
    "train_augmentation": "TrivialAugmentWide",
    "seed": 97531
}
2025-11-24 09:23:56,682 - __main__ - INFO - S

### Option B: 전체 학습 10 epochs (~3-4시간)

⚠️ GPU 사용 시 약 3-4시간 소요됩니다.

In [8]:
# 10 epochs GPU 학습
!python mouse_facial_expressions/models/train_task1_baseline_model.py \
    --epochs 10 \
    --dataset_version "1.1"

2025-11-24 10:03:55,013 - __main__ - INFO - beginning model run train_task1_baseline_model.py
/opt/anaconda3/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025-11-24 10:03:55,428 - __main__ - INFO - Started mlflow crossvalidation run 3782f4243ec24491adc3a88a9f60eb11
2025-11-24 10:03:55,429 - __main__ - INFO - Logging parameters
{
    "epochs": 10,
    "dataset_version": "1.1",
    "model": 10,
    "learning_rate": 0.01,
    "train_batch_size": 20,
    "test_batch_size": 50,
    "warmup_steps": 0,
    "momentum": 0.9,
    "weight_decay": 0.0001,
    "warmup_decay": 0.0001,
    "label_smoothing": 0.1,
    "train_augmentation": "TrivialAugmentWide",
    "seed": 97531
}
2025-11-24 10:03:55,460 - __main__ - INFO - 

## Step 6: 학습 모니터링

In [9]:
# 체크포인트 확인
import os
import glob

checkpoint_dir = 'models/checkpoints'
if os.path.exists(checkpoint_dir):
    checkpoints = glob.glob(f'{checkpoint_dir}/*.ckpt')
    if checkpoints:
        print(f"✅ 저장된 체크포인트 ({len(checkpoints)}개):")
        !ls -lht models/checkpoints/ | head -10
    else:
        print("아직 체크포인트가 없습니다.")
else:
    print("체크포인트 디렉토리가 아직 생성되지 않았습니다.")

✅ 저장된 체크포인트 (19개):
total 6325632
-rw-------  1 minakang  staff   163M Nov 24 15:15 task1-fold7-epoch=4-val_acc=1.00.ckpt
-rw-------  1 minakang  staff   163M Nov 24 14:24 task1-fold6-epoch=2-val_acc=0.98.ckpt
-rw-------  1 minakang  staff   163M Nov 24 13:56 task1-fold5-epoch=6-val_acc=0.98.ckpt
-rw-------  1 minakang  staff   163M Nov 24 13:27 task1-fold4-epoch=9-val_acc=0.97.ckpt
-rw-------  1 minakang  staff   163M Nov 24 12:42 task1-fold3-epoch=8-val_acc=0.97.ckpt
-rw-------  1 minakang  staff   163M Nov 24 11:49 task1-fold2-epoch=5-val_acc=0.99.ckpt
-rw-------  1 minakang  staff   163M Nov 24 11:04 task1-fold1-epoch=4-val_acc=0.98.ckpt
-rw-------  1 minakang  staff   163M Nov 24 10:07 task1-fold0-epoch=0-val_acc=1.00-v2.ckpt
-rw-------  1 minakang  staff   163M Nov 24 10:02 task1-fold7-epoch=0-val_acc=1.00.ckpt


In [10]:
# 학습 진행 중인지 확인
!ps aux | grep train_task1 | grep -v grep

In [11]:
# MLflow 실험 결과 확인
import mlflow
import os

mlflow.set_tracking_uri("file://" + os.path.join(os.getcwd(), "mlruns"))

client = mlflow.tracking.MlflowClient()
experiments = client.search_experiments()

if experiments:
    print("MLflow 실험:")
    for exp in experiments:
        print(f"\n실험 ID: {exp.experiment_id}")
        print(f"이름: {exp.name}")
        
        runs = client.search_runs(exp.experiment_id, max_results=3)
        if runs:
            print(f"\n최근 실행 ({len(runs)}개):")
            for run in runs:
                print(f"  - Run ID: {run.info.run_id[:8]}...")
                print(f"    상태: {run.info.status}")
                if run.data.metrics:
                    metrics = dict(list(run.data.metrics.items())[:3])
                    print(f"    메트릭: {metrics}")
else:
    print("아직 MLflow 실험이 없습니다.")

MLflow 실험:

실험 ID: 0
이름: Default


/opt/anaconda3/lib/python3.13/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)



최근 실행 (3개):
  - Run ID: ad876f1a...
    상태: FINISHED
    메트릭: {'val_loss': 0.25352540910243987, 'learning_rate': 0.0, 'train_acc': 0.9965000152587891}
  - Run ID: 290ee34a...
    상태: FINISHED
    메트릭: {'val_loss': 0.2737993973493576, 'learning_rate': 0.0, 'train_acc': 0.9977999925613403}
  - Run ID: cb841912...
    상태: FINISHED
    메트릭: {'val_loss': 0.3134676992893219, 'learning_rate': 0.0, 'train_acc': 0.9980000257492065}


## Step 7: 결과 확인

In [13]:
# 최고 성능 모델 찾기
import glob
import re
import os

checkpoints = glob.glob('models/checkpoints/*.ckpt')
if checkpoints:
    best_acc = 0
    best_ckpt = None
    
    print("저장된 체크포인트:")
    for ckpt in sorted(checkpoints):
        basename = os.path.basename(ckpt)
        match = re.search(r'val_acc=(\d+\.\d+)', basename)
        if match:
            acc = float(match.group(1))
            size_mb = os.path.getsize(ckpt) / (1024 * 1024)
            print(f"  - {basename}")
            print(f"    정확도: {acc:.2%}, 크기: {size_mb:.1f} MB")
            if acc > best_acc:
                best_acc = acc
                best_ckpt = ckpt
    
    if best_ckpt:
        print(f"\n🏆 최고 성능 모델:")
        print(f"   파일: {os.path.basename(best_ckpt)}")
        print(f"   Validation Accuracy: {best_acc:.2%}")
else:
    print("아직 체크포인트가 없습니다.")

저장된 체크포인트:
  - task1-fold0-epoch=0-val_acc=1.00-v1.ckpt
    정확도: 100.00%, 크기: 162.6 MB
  - task1-fold0-epoch=0-val_acc=1.00-v2.ckpt
    정확도: 100.00%, 크기: 162.6 MB
  - task1-fold0-epoch=0-val_acc=1.00.ckpt
    정확도: 100.00%, 크기: 162.6 MB
  - task1-fold1-epoch=0-val_acc=0.84-v1.ckpt
    정확도: 84.00%, 크기: 162.6 MB
  - task1-fold1-epoch=0-val_acc=0.84.ckpt
    정확도: 84.00%, 크기: 162.6 MB
  - task1-fold1-epoch=4-val_acc=0.98.ckpt
    정확도: 98.00%, 크기: 162.6 MB
  - task1-fold2-epoch=0-val_acc=0.94-v1.ckpt
    정확도: 94.00%, 크기: 162.6 MB
  - task1-fold2-epoch=0-val_acc=0.94.ckpt
    정확도: 94.00%, 크기: 162.6 MB
  - task1-fold2-epoch=5-val_acc=0.99.ckpt
    정확도: 99.00%, 크기: 162.6 MB
  - task1-fold3-epoch=0-val_acc=0.93.ckpt
    정확도: 93.00%, 크기: 162.6 MB
  - task1-fold3-epoch=8-val_acc=0.97.ckpt
    정확도: 97.00%, 크기: 162.6 MB
  - task1-fold4-epoch=0-val_acc=0.95.ckpt
    정확도: 95.00%, 크기: 162.6 MB
  - task1-fold4-epoch=9-val_acc=0.97.ckpt
    정확도: 97.00%, 크기: 162.6 MB
  - task1-fold5-epoch=0-val_acc=0.97.c

## 💡 GPU 학습 팁

### MPS (Metal Performance Shaders) 특징
- ✅ **Apple Silicon GPU 가속**: M1/M2/M3 칩의 GPU 활용
- ✅ **2-3배 빠른 학습**: CPU 대비 훨씬 빠름
- ✅ **자동 감지**: `accelerator='auto'`로 설정하여 자동으로 GPU 사용
- ⚠️ **메모리 제한**: 통합 메모리 사용 (RAM과 공유)
- ⚠️ **float64 미지원**: float32만 지원 (이미 수정됨)

### 성능 비교
| 항목 | CPU | MPS (GPU) |
|------|-----|----------|
| 1 epoch | 40-50분 | 15-20분 ⚡ |
| 10 epochs | 7-8시간 | 3-4시간 ⚡ |
| 배치 크기 | 20 | 20-40 (메모리에 따라) |

### 학습 로그에서 GPU 사용 확인
학습 시작 시 다음과 같이 표시되면 GPU를 사용하고 있습니다:
```
GPU available: True (mps), used: True
```

### 문제 해결

**❌ TypeError: Cannot convert a MPS Tensor to float64**
- ✅ 이미 수정되었습니다! 모든 데이터가 float32로 변환됩니다.

**MPS 에러 발생 시**
```bash
# 환경 변수 설정 (터미널에서)
export PYTORCH_ENABLE_MPS_FALLBACK=1
```

**메모리 부족 에러**
- 배치 크기를 줄이세요 (20 → 10)
- 다른 앱을 종료하여 메모리 확보

**학습 속도가 느림**
- Activity Monitor에서 GPU 사용률 확인
- 백그라운드 앱 종료
- 전원 어댑터 연결 확인 (배터리 모드에서는 성능 제한)

### MLflow UI 실행
터미널에서:
```bash
cd /Users/minakang/Desktop/mouse-facial-expressions-2023-main
mlflow ui
```
브라우저에서 http://localhost:5000 접속

### 학습 중단하기
- Jupyter 메뉴: Kernel → Interrupt
- 또는 셀 실행 중 ■ 버튼 클릭